In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install langchain-core langchain-openai pydantic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.3 MB/s eta 0:00:00


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from sklearn.metrics import classification_report
import time
import os
from tqdm import tqdm

In [ ]:
path = '/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/total_news_esg_filtered.csv'
df_trainning = pd.read_csv(path)
df_trainning

<ipython-input-5-60e0852d7363>:2: DtypeWarning: Columns (5,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_trainning = pd.read_csv(path)


,id,Publication Date,title,content,source,sentiment_body,sentiment_body_score,company,sector,url,ticker,description,text,esg_pred
0,1.0,2024-12-09 08:41:30+00:00,Call of Duty: Black Ops 6 Review,The last year or so wasn’t the kindest to Call...,Eggplante,positive,0.98,activision,Communication,NaN,ATVI,NaN,Call of Duty: Black Ops 6 Review. The last yea...,0
1,2.0,2024-07-23 17:13:37+00:00,Xbox Celebrates the Release of Modern Warfare ...,"Recently, many rumors were running on the inte...",Cinelinx,positive,0.90,activision,Communication,NaN,ATVI,NaN,Xbox Celebrates the Release of Modern Warfare ...,0
2,3.0,2024-07-18 14:19:34+00:00,How ToxMod's AI impacted toxicity in Call of D...,It's no secret Call of Duty has toxic players....,DNyuz,neutral,0.48,activision,Communication,NaN,ATVI,NaN,How ToxMod's AI impacted toxicity in Call of D...,1
3,4.0,2024-07-18 14:04:50+00:00,How ToxMod's AI impacted toxicity in Call of D...,It's no secret Call of Duty has toxic players....,Allusanewshub,neutral,0.54,activision,Communication,NaN,ATVI,NaN,How ToxMod's AI impacted toxicity in Call of D...,1
4,5.0,2024-05-27 07:03:20+00:00,"Meta, Activision Sued by Families of Uvalde Sc...",In the wake of the tragic shooting at Robb Ele...,Tech Times,neutral,0.92,activision,Communication,NaN,ATVI,NaN,"Meta, Activision Sued by Families of Uvalde Sc...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070256,NaN,2025-03-21 16:21:09,Nike shares tumble as tariff concerns shake in...,"Shares in NIKE, Inc. (NYSE: NKE) are falling i...",Fast Company,NaN,NaN,nike,NaN,https://www.fastcompany.com/91303743/nike-shar...,NKE,NaN,Nike shares tumble as tariff concerns shake in...,1
1070257,NaN,2025-03-21 20:20:23,Making Sense of Early Q1 Earnings Reports,We have been seeing some of the early Q1 resul...,Zacks Investment Research,NaN,NaN,nike,NaN,https://www.zacks.com/commentary/2433762/makin...,NKE,NaN,Making Sense of Early Q1 Earnings Reports. We ...,0
1070258,NaN,2025-03-22 07:30:00,"Why Is Nike Stock Falling, and Should Investor...",Nike's (NKE -5.37%) sales are declining in eve...,The Motley Fool,NaN,NaN,nike,NaN,https://www.fool.com/investing/2025/03/22/why-...,NKE,NaN,"Why Is Nike Stock Falling, and Should Investor...",0
1070259,NaN,2025-03-22 11:00:00,Will Nike Investors' Frustrations End Anytime ...,"If you're a Nike (NKE -5.37%) investor, it wou...",The Motley Fool,NaN,NaN,nike,NaN,https://www.fool.com/investing/2025/03/22/will...,NKE,NaN,Will Nike Investors' Frustrations End Anytime ...,0


In [ ]:
df_esg = df_trainning[df_trainning['esg_pred'] == 1].copy()

In [ ]:
os.environ["OPENAI_API_KEY"] = "xxxxx"

In [ ]:
news_sample = df_esg.sample(n=8000, random_state=33).reset_index(drop=True)

In [ ]:
class ESGCategory(BaseModel):
    category: str = Field(..., description="ESG category of the news. One of: 'Environmental', 'Social', or 'Governance'")

In [ ]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
You are an ESG analyst.

Classify the following news article into one of the ESG categories:
- Environmental (e.g., emissions, energy, pollution, biodiversity, sustainability)
- Social (e.g., labor practices, community, diversity, equality, employee well-being)
- Governance (e.g., corporate ethics, transparency, board structure, regulation)

Always choose one category only: "Environmental", "Social", or "Governance".

Text:
{input}
"""
)

In [ ]:
model = ChatOpenAI(model="gpt-4o-mini", temperature=0).with_structured_output(ESGCategory)
tagging_chain = tagging_prompt | model

In [ ]:
df_esg

,id,Publication Date,title,content,source,sentiment_body,sentiment_body_score,company,sector,url,ticker,description,text,esg_pred
2,3.0,2024-07-18 14:19:34+00:00,How ToxMod's AI impacted toxicity in Call of D...,It's no secret Call of Duty has toxic players....,DNyuz,neutral,0.48,activision,Communication,NaN,ATVI,NaN,How ToxMod's AI impacted toxicity in Call of D...,1
3,4.0,2024-07-18 14:04:50+00:00,How ToxMod's AI impacted toxicity in Call of D...,It's no secret Call of Duty has toxic players....,Allusanewshub,neutral,0.54,activision,Communication,NaN,ATVI,NaN,How ToxMod's AI impacted toxicity in Call of D...,1
4,5.0,2024-05-27 07:03:20+00:00,"Meta, Activision Sued by Families of Uvalde Sc...",In the wake of the tragic shooting at Robb Ele...,Tech Times,neutral,0.92,activision,Communication,NaN,ATVI,NaN,"Meta, Activision Sued by Families of Uvalde Sc...",1
7,8.0,2024-04-02 15:34:29+00:00,Federal racial discrimination lawsuit against ...,"'In Tesla's narrative, the agencies learned no...",Head Topics,negative,0.41,activision,Communication,NaN,ATVI,NaN,Federal racial discrimination lawsuit against ...,1
10,11.0,2024-01-24 03:28:56+00:00,Activision Will Pay $50 Million to Settle Work...,Activision Blizzard will pay roughly $50 milli...,Mytimesnow,positive,0.64,activision,Communication,NaN,ATVI,NaN,Activision Will Pay $50 Million to Settle Work...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070247,NaN,2025-03-21 10:50:51,Nike is starting to show green-shoots with inn...,"Andrea Andreeva, Piper Sandler analyst, joins ...",CNBC Television,NaN,NaN,nike,NaN,https://www.youtube.com/watch?v=yrwUTpiLvsI,NKE,NaN,Nike is starting to show green-shoots with inn...,1
1070249,NaN,2025-03-21 11:19:37,"Nike Stock Plummets on Dismal Forecast, Bear N...",Brushing off a fiscal third-quarter earnings a...,Schaeffers Research,NaN,NaN,nike,NaN,https://www.schaeffersresearch.com/content/new...,NKE,NaN,"Nike Stock Plummets on Dismal Forecast, Bear N...",1
1070250,NaN,2025-03-21 11:20:15,Nike Turnaround Hits Snags Amid Inventory Rese...,Nike's turnaround effort is facing challenges ...,Bloomberg Markets and Finance,NaN,NaN,nike,NaN,https://www.youtube.com/watch?v=T4MRIaMz2ks,NKE,NaN,Nike Turnaround Hits Snags Amid Inventory Rese...,1
1070253,NaN,2025-03-21 12:31:54,Why Nike Stock Got Tripped Up Today,Shares of Nike (NKE -5.16%) fell Friday mornin...,The Motley Fool,NaN,NaN,nike,NaN,https://www.fool.com/investing/2025/03/21/why-...,NKE,NaN,Why Nike Stock Got Tripped Up Today. Shares of...,1


In [ ]:
categories = []

start = time.time()

for i, row in tqdm(news_sample.iterrows(), total=len(news_sample)):
    text = row['text']
    try:
        result = tagging_chain.invoke({"input": text})
        categories.append(result.category)
    except Exception as e:
        categories.append(None)

print("Tiempo total:", round(time.time() - start, 2), "segundos")
news_sample['esg_category'] = categories

100%|██████████| 8000/8000 [1:10:05<00:00,  1.90it/s]

⏱️ Tiempo total: 4205.55 segundos


In [ ]:
news_sample['esg_category'].value_counts()

,count
esg_category,
Governance,3448
Social,2872
Environmental,1680


In [ ]:
news_sample.to_csv('/content/drive/MyDrive/MIAX/TRABAJO FINAL DE MASTER DEFINITIVO/sample_e_s_g.csv', index=False)